In [1]:
import numpy as np
import json
import pickle

In [2]:
filename = '/mnt/glove.6B.50d.txt'

def loadGloVe(filename):
    vocab = dict()
    embd = []
    file = open(filename,'r')
    for index, line in enumerate(file.readlines()):
        row = line.strip().split(' ')
        vocab[index] = row[0]
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd

vocab, embd = loadGloVe(filename)
embedding = np.asarray(embd)

Loaded GloVe!


In [3]:
np.save("word_embedding_matrix", embedding)

In [4]:
with open('vocabulary.pickle', 'wb') as handle:
    pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

Load training set

In [2]:
file_name = "training.json"
with open(file_name) as json_data:
    training_set = json.load(json_data)

Load documents

In [3]:
file_name = "documents.json"
with open(file_name) as json_data:
    documents = json.load(json_data)

Preprocess the output

In [4]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
training_data = []

for i in training_set:
    curr_dct = dict()
    
    q = i["question"].lower()
    curr_dct["question"] = [lemmatizer.lemmatize(i) for i in word_tokenize(q)]
    
    ans = i["text"].lower()
    tokenized_ans = [lemmatizer.lemmatize(i) for i in word_tokenize(ans)]
    
    docid = i["docid"]
    answer_para = i["answer_paragraph"]
    
    c = documents[docid]["text"][answer_para].lower()
    tokenized_c = [lemmatizer.lemmatize(i) for i in word_tokenize(c)]
    
    curr_dct["context"] = tokenized_c
    
    # find the position that answer occur
    for pos, i in enumerate(tokenized_c):
        ac_list = zip(tokenized_ans, tokenized_c[pos:])
        if False in [True if i in j else False for i, j in ac_list]:
            continue
        else:
            start = [0] * len(tokenized_c)
            end = [0] * len(tokenized_c)
            start[pos] = 1
            end[pos + len(tokenized_ans) - 1] = 1
            curr_dct["start"] = start
            curr_dct["end"] = end
            break
            
    training_data.append(curr_dct)

In [6]:
with open('training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)